In [1]:
import numpy as np

In [2]:
def matrix_dot(A, B):
    if type(A[0]) != list:
        A = [A]

    if len(A[0]) != len(B):
        raise ValueError("행렬 곱 연산이 불가능합니다.")

    result = [[0 for _ in range(len(B[0]))] for _ in range(len(A))]
    for i in range(len(A)):
        for j in range(len(B[0])):
            dot_product = 0
            for k in range(len(A[0])):
                dot_product += A[i][k] * B[k][j]
            result[i][j] = dot_product
    return result

In [3]:
##########주어짐###########
#입력
X=[[i+0.1,i] for i in range(1000)] #한 시점에 특징이 2가지인 데이터 1000개
#print("X:",np.array(X).shape)
#print(X)
#배치 구현 <- simple_lstm 참고하여 바꿔야
BatchSize = int(input("배치 사이즈:"))
X=X[:BatchSize]
#print("X:",np.array(X).shape)
#print(X)
timeSize = int(input("한번에 고려할 시점 수:")) #한번에 고려할 시점의 개수
newX=[]
for i in range(len(X)-timeSize+1): # [0 1 2 3 4]를 2시점 씩 고려하면 [0 1] [1 2] [2 3] [3 4] 총 4가지가 되므로 
    smallX=[]
    for j in range(timeSize):
        smallX.append(X[i+j])
    newX.append(smallX)
X = newX
print("X:",np.array(X).shape)
print(X)
input_len = len(X)  #데이터의 개수만큼 target과 예측값이 있어야 하므로

#target
T=[[i//10] for i in range(input_len)] # 임의로 첫번째 값을 10으로 나눈 몫으로 정함 , 타겟이 마지막 층이라 원소가 1개인 target 값이 input_len만큼있음
print("T:",np.array(T).shape)
print(T)

#셔플수행하면 좋으나 그냥지나감

X: (98, 3, 2)
[[[0.1, 0], [1.1, 1], [2.1, 2]], [[1.1, 1], [2.1, 2], [3.1, 3]], [[2.1, 2], [3.1, 3], [4.1, 4]], [[3.1, 3], [4.1, 4], [5.1, 5]], [[4.1, 4], [5.1, 5], [6.1, 6]], [[5.1, 5], [6.1, 6], [7.1, 7]], [[6.1, 6], [7.1, 7], [8.1, 8]], [[7.1, 7], [8.1, 8], [9.1, 9]], [[8.1, 8], [9.1, 9], [10.1, 10]], [[9.1, 9], [10.1, 10], [11.1, 11]], [[10.1, 10], [11.1, 11], [12.1, 12]], [[11.1, 11], [12.1, 12], [13.1, 13]], [[12.1, 12], [13.1, 13], [14.1, 14]], [[13.1, 13], [14.1, 14], [15.1, 15]], [[14.1, 14], [15.1, 15], [16.1, 16]], [[15.1, 15], [16.1, 16], [17.1, 17]], [[16.1, 16], [17.1, 17], [18.1, 18]], [[17.1, 17], [18.1, 18], [19.1, 19]], [[18.1, 18], [19.1, 19], [20.1, 20]], [[19.1, 19], [20.1, 20], [21.1, 21]], [[20.1, 20], [21.1, 21], [22.1, 22]], [[21.1, 21], [22.1, 22], [23.1, 23]], [[22.1, 22], [23.1, 23], [24.1, 24]], [[23.1, 23], [24.1, 24], [25.1, 25]], [[24.1, 24], [25.1, 25], [26.1, 26]], [[25.1, 25], [26.1, 26], [27.1, 27]], [[26.1, 26], [27.1, 27], [28.1, 28]], [[27.1, 27], 

In [4]:
outputQ = input("LSTM 한 층의 입력하나요(Y/N)?:") #층이 하나면 그냥 target의 종류, 층이 여러개면 정해줘야 함. 현재는 단일 모듈이므로 target의 종류로 정함
count_list=[] # 타겟의 종류 수 카운트 => layer output의 크기를 정하기 위해서(위의 ? 정함) lstm 층이 여러개면 lstm 모듈마다 output_size 정해야
cnt = 0
for now in T:
    if now in count_list:
        pass
    else:
        cnt += 1
        count_list.append(now)
output_size = cnt #w의 output 크기
if(outputQ == "N"):
    output_size = int(input("output_size:")) 
print(output_size)

10


In [5]:
##########작성###########
#state초기화
Y = [[[0 for i in range(output_size)] for i in range(timeSize)] for _ in range(input_len)] # 실제 값은 X의 입력 만큼 있어야 하므로
C = [[[0 for i in range(output_size)] for i in range(timeSize)] for _ in range(input_len)] # X은 시점 수 만큼 있어야 하므로
print("Y:",np.array(Y).shape)
print(Y)
print("C:",np.array(C).shape)
print(C)

Y: (98, 3, 10)
[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0,

In [6]:
#parameter초기화
input_feature = len(X[0][0]) # 한 시점에서 데이터의 특징 수 <= X와 w의 행렬곱을 수행해야하므로 X가 (시점수,2)이므로 w는 (2,?)가 되야함
print(input_feature)

2


In [7]:
#parameter 초기화
import random
#가중치 초기화: 가우시안 초기화 방법,  He 초기화 (He Initialization) 또는 Xavier 초기화 (Glorot Initialization)도 존재
w_matrix = []
v_matrix = []
b_matrix = []
for _ in range(4): ## 망각 게이트,입력 게이트,새로운 기억, 출력 게이트 총 4개에서 업데이트 해야함
    w0=[[random.gauss(0,1) for i in range(output_size)] for j in range(input_feature)] # X는 (시점 수, 특징 수)이므로 w 는 (특징수, output_size) 
    #print(np.array(w0).shape)
    v0=[[random.gauss(0,1) for i in range(output_size)]for j in range(output_size)]  # Y는 (시점 수, 특징 수)이므로 w 는 (특징수, output_size)
    #print(np.array(v0).shape)
    b0=[[random.gauss(0,1) for i in range(output_size)]for j in range(timeSize)] 
    #print(np.array(b0).shape)
    w_matrix.append(w0)
    v_matrix.append(v0)
    b_matrix.append(b0)
print('X')
print(np.array(X).shape)
print('w_matrix')
print(np.array(w_matrix).shape)
print('Y')
print(np.array(Y).shape)
print('v_matrix')
print(np.array(v_matrix).shape)
################################
print('b_matrix')
print(np.array(b_matrix).shape)


X
(98, 3, 2)
w_matrix
(4, 2, 10)
Y
(98, 3, 10)
v_matrix
(4, 10, 10)
b_matrix
(4, 3, 10)


In [8]:
now = 1
nowtime = 1
print("==================")
print("Xw")
print(np.dot(X[now],w_matrix[0]).shape)
print(np.dot(X[now],w_matrix[0]))
print(np.array(matrix_dot(X[nowtime],w_matrix[0])).shape)
print(np.array(matrix_dot(X[nowtime],w_matrix[0])))


Xw
(3, 10)
[[-0.65984525 -1.87372792  0.81158713  0.8739938   1.723551   -0.8692907
   1.23009099  2.55637033  2.00868694  0.52777722]
 [-1.2877655  -3.72895324  1.66923244  1.70873353  3.31777075 -1.65473913
   2.50311657  4.98851681  3.94107769  0.96329525]
 [-1.91568574 -5.58417857  2.52687775  2.54347326  4.9119905  -2.44018756
   3.77614216  7.42066329  5.87346844  1.39881327]]
(3, 10)
[[-0.65984525 -1.87372792  0.81158713  0.8739938   1.723551   -0.8692907
   1.23009099  2.55637033  2.00868694  0.52777722]
 [-1.2877655  -3.72895324  1.66923244  1.70873353  3.31777075 -1.65473913
   2.50311657  4.98851681  3.94107769  0.96329525]
 [-1.91568574 -5.58417857  2.52687775  2.54347326  4.9119905  -2.44018756
   3.77614216  7.42066329  5.87346844  1.39881327]]


In [9]:
print("==================")
print("Yv")
print(np.dot(Y[now],v_matrix[0]).shape)
print(np.dot(Y[now],v_matrix[0]))
print(np.array(matrix_dot(Y[nowtime],v_matrix[0])).shape)
print(np.array(matrix_dot(Y[nowtime],v_matrix[0])))

Yv
(3, 10)
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
(3, 10)
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [15]:
def element_sum(arr1, arr2, arr3):
    if len(arr1) != len(arr2) or len(arr1) != len(arr3) or len(arr1[0]) != len(arr2[0]) or len(arr1[0]) != len(arr3[0]):
        raise ValueError("Input arrays must have the same shape")
    print(np.array(arr1))
    print(np.array(arr2))
    print(np.array(arr3))
    result = []
    for i in range(len(arr1)):
        row = []
        for j in range(len(arr1[0])):
            row.append(arr1[i][j] + arr2[i][j] + arr3[i][j])
        result.append(row)
    
    return result

In [71]:
print("==================")
print("Xw+Yv+b")
print((np.dot(X[now],w_matrix[0])+np.dot(Y[now],v_matrix[0])+b_matrix[0]).shape)
print((np.dot(X[now],w_matrix[0])+np.dot(Y[now],v_matrix[0])+b_matrix[0]))
Xw = matrix_dot(X[nowtime],w_matrix[0])
Yv = matrix_dot(Y[nowtime],v_matrix[0])
Xw_Yv_b = element_sum(Xw,Yv,b_matrix[0])
print(np.array(Xw_Yv_b).shape)
print(np.array(Xw_Yv_b))
print("==================")
fCalculate = element_sum(X[nowtime],w_matrix[0],Y[nowtime],v_matrix[0],b_matrix[0])
print(np.array(fCalculate).shape)
print(np.array(fCalculate))
#망각gate

Xw+Yv
(5, 10)
[[ 0.0340596   1.0134602   1.10394348  0.12670858 -0.56506761 -0.34291005
   0.21804016 -0.27130441 -1.89470356 -0.48843086]
 [ 0.16764903  1.89449723  2.25469923  0.19617977 -1.15065325 -0.65145062
   0.39842129 -0.55074563 -3.66278478 -0.97443444]
 [ 0.30123845  2.77553427  3.40545497  0.26565096 -1.7362389  -0.95999119
   0.57880242 -0.83018685 -5.43086601 -1.46043801]
 [ 0.43482788  3.6565713   4.55621072  0.33512215 -2.32182455 -1.26853177
   0.75918355 -1.10962807 -7.19894723 -1.94644159]
 [ 0.5684173   4.53760833  5.70696647  0.40459334 -2.90741019 -1.57707234
   0.93956468 -1.38906929 -8.96702846 -2.43244516]]
(5, 10)
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


Xw+Yv+b


ValueError: operands could not be broadcast together with shapes (5,10) (10,10) 

In [ ]:
def activation(function,x):
    e = 2.718281828459045 #자연상수:2.71828182845904523536
    result = [0 for i in range(len(x))]
    if(function == 'Sigmoid'):
            for i in range(len(x)):
                result[i] = (e**x[i])/((e**x[i])+1) #C언어에서는 **이 없어 **도 구현
    if(function == 'Tanh'):
            for i in range(len(x)):
                result[i] = ((e**x[i]) - (e**(-1*x[i]))) / ((e**x[i]) + (e**(-1*x[i]))) 
    if(function == 'ReLU'):
        if (x<=0):
            return 0
        return x
    return [result]

In [115]:
fCalculate = element_sum(X[nowtime],w_matrix[0],Y[nowtime],v_matrix[0],b_matrix[0])
print(fCalculate)
print(np.array(fCalculate).shape)

[[-0.27538637550964684, -1.2539284818163428, 0.3827067952578748, -0.9261470773291458, -0.385803671503714, -0.24023050445953265, -3.7874990976472156, 0.40784399958308704, -0.8514779734768061, -0.9549174627177401]]
(1, 10)


In [120]:
#망각gate
nowtime = 1
fCalculate = element_sum(X[nowtime],w_matrix[0],Y[nowtime],v_matrix[0],b_matrix[0])
fGate = matrix_dot(C[nowtime], activation("Sigmoid",fCalculate[0])) # X는 시점 t, Y는 시점 t-1
C[nowtime] = D1vectorsum(fGate[0])
#print(C)
print(np.array(activation("Sigmoid",fCalculate[0])).shape)
#입력gate
iCalculate = element_sum(X[nowtime],w_matrix[1],Y[nowtime],v_matrix[1],b_matrix[1])
nCalculate = element_sum(X[nowtime],w_matrix[2],Y[nowtime],v_matrix[2],b_matrix[2])
print(np.array(activation("Sigmoid",iCalculate[0])).shape) # 
print(np.array(activation("Tanh",nCalculate[0])).shape)
#iGate = matrix_dot(activation("Sigmoid",iCalculate[0]),activation("Tanh",nCalculate[0]))
"""
iGate = C +(activation("Sigmoid",iCalaulate[0]) * activation("Tanh",nCalaulate[0]))
C = iGate
#출력gate
oCalculate = element_sum(X[nowtime],w_matrix[3],Y[nowtime],v_matrix[3],b_matrix[3])
oGate = activation("Sigmoid",oCalculate) * activation("Tanh",C) 
Y = oGate
"""

(1, 10)
(1, 10)
(1, 10)


'\niGate = C +(activation("Sigmoid",iCalaulate[0]) * activation("Tanh",nCalaulate[0]))\nC = iGate\n#출력gate\noCalculate = element_sum(X[nowtime],w_matrix[3],Y[nowtime],v_matrix[3],b_matrix[3])\noGate = activation("Sigmoid",oCalculate) * activation("Tanh",C) \nY = oGate\n'

In [68]:
print(C)

[[0], [0.0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0]]


In [44]:
print(C[1])

[0.0]


In [48]:
C[1]= [2]

In [49]:
print(C)

[[0.0], [2], [0.0], [0.0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0]]
